# Overture Maps - Places Query
Load Overture Maps data into DuckDB and query specific categories in Tennessee

## Setup Instructions
Follow the official Overture Maps documentation approach using DuckDB with httpfs extension.

Reference: https://docs.overturemaps.org/getting-data/duckdb/

In [1]:
import duckdb
import pandas as pd

In [2]:
# Create a DuckDB connection
con = duckdb.connect()

# Install and load the httpfs and spatial extensions
con.execute("INSTALL httpfs")
con.execute("LOAD httpfs")
con.execute("INSTALL spatial")
con.execute("LOAD spatial")

# Configure for anonymous S3 access
con.execute("SET s3_region='us-west-2'")

print("DuckDB connection established and extensions loaded")

DuckDB connection established and extensions loaded


In [3]:
# Load Overture Maps 'places' data
# Using the official Overture Maps approach with latest release

# Latest release: 2026-01-21.0
places_path = "s3://overturemaps-us-west-2/release/2026-01-21.0/theme=places/type=place/*"

print(f"Loading Overture places data from: {places_path}")

# Create view using read_parquet as per Overture documentation
con.execute(f"""
CREATE OR REPLACE VIEW places AS 
SELECT * FROM read_parquet('{places_path}', 
                            filename=true, 
                            hive_partitioning=1)
""")

print("✓ Overture places data loaded successfully")

# Quick verification
sample = con.execute("SELECT COUNT(*) OVER () as total FROM places LIMIT 1").fetchone()
print("Places view created and ready to query")

Loading Overture places data from: s3://overturemaps-us-west-2/release/2026-01-21.0/theme=places/type=place/*
✓ Overture places data loaded successfully
Places view created and ready to query


In [4]:
# Explore the categories structure to understand the schema
structure_query = """
SELECT 
    categories,
    names,
    addresses
FROM places 
WHERE categories IS NOT NULL
LIMIT 5
"""

sample_data = con.execute(structure_query).fetchdf()
print("Sample categories structure:")
print(sample_data['categories'].head())
print("\nSample names structure:")
print(sample_data['names'].head())
print("\nSample addresses structure:")  
print(sample_data['addresses'].head())

Sample categories structure:
0    {'primary': 'information_technology_company', ...
1    {'primary': 'internet_marketing_service', 'alt...
2    {'primary': 'physical_therapy', 'alternate': [...
3    {'primary': 'health_and_medical', 'alternate':...
4             {'primary': 'school', 'alternate': None}
Name: categories, dtype: object

Sample names structure:
0    {'primary': 'Tincorp Pa TI', 'common': None, '...
1    {'primary': 'Media Ozone', 'common': None, 'ru...
2    {'primary': 'Sara Couto - Intervenção Terapêut...
3    {'primary': 'แหลมใหญ่แล็บ คลินิกเทคนิคการแพทย์...
4    {'primary': 'Chittagong Victory National Schoo...
Name: names, dtype: object

Sample addresses structure:
0    [{'freeform': 'Avenida Calle 22-86', 'locality...
1    [{'freeform': 'rue de l'Indépendance 17 bloc',...
2    [{'freeform': 'Rua João Marcos (escritor)', 'l...
3    [{'freeform': 'แหลมใหญ่แล็บ คลินิกเทคนิคการแพท...
4    [{'freeform': '409,S,S Khaled Road, Jamal Khan...
Name: addresses, dtype: object


In [5]:
# Alternative: Filter by Tennessee bounding box for better performance
# Tennessee bbox: approximately [-90.3, 34.9, -81.6, 36.7]
# This approach is more efficient for large datasets

tn_bbox_query = """
SELECT 
    id, 
    names.primary as name, 
    categories.primary as category, 
    addresses[1].region as state,
    addresses[1].locality as city,
    geometry
FROM places
WHERE categories.primary IN (
    'mobile_home_park',
    'homeless_shelter', 
    'social_facility', 
    'nursing_home', 
    'assisted_living'
)
AND bbox.xmin >= -90.3 
AND bbox.xmax <= -81.6
AND bbox.ymin >= 34.9
AND bbox.ymax <= 36.7
"""

# Uncomment to use bbox filtering instead:
# df = con.execute(tn_bbox_query).fetchdf()
# print(f"Found {len(df):,} places in Tennessee (bbox filtered)")

In [6]:
# Query specific categories in Tennessee
# Note: Using categories.primary (not .main) based on Overture schema

query = """
SELECT 
    id, 
    names.primary as name, 
    categories.primary as category, 
    addresses[1].region as state,
    addresses[1].locality as city,
    geometry
FROM places
WHERE categories.primary IN (
    'mobile_home_park',
    'homeless_shelter', 
    'social_facility', 
    'nursing_home', 
    'assisted_living'
)
AND addresses[1].region = 'TN'
"""

# Execute query and convert to pandas DataFrame
df = con.execute(query).fetchdf()

print(f"Found {len(df):,} places in Tennessee matching the criteria")
df.head(10)

Found 175 places in Tennessee matching the criteria


,id,name,category,state,city,geometry
0,e05479f8-c1ab-4d58-9081-b571154f279d,Candlelight,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,75d242ff-9944-494a-b1c4-3d9407c47972,Wheel Estates,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,98c085eb-0449-462e-ada1-8d9af1851092,Candlewood,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,27471146-80fa-405a-a7d7-6ed90d8548f8,Housing The Homeless - Helps,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,70fc56d2-7c5e-4759-8240-1fd9bfc663ed,Calvary Rescue Mission,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,31af2467-638b-4c12-a809-3efa567a9079,Barron Heights Transitional Center,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,b9c946cf-6eb3-475a-9936-33c666d166cd,Alpha Omega Veterans Services - Vinton,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,a33313a3-f6a3-43e2-9550-c5c7b56cfe10,Dorothy Day House Of Hospitality,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,25898f67-868f-4417-9420-f4811013ac1f,Dorothy Day House,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,ff70aa3c-6dac-4dfb-bafa-f0bec89a0ed9,Room In The Inn,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [ ]:
# Check what categories actually exist in Tennessee data
# This helps verify if our category names are correct

explore_categories = """
SELECT 
    categories.primary as category,
    COUNT(*) as count
FROM places
WHERE addresses[1].region = 'TN'
AND categories.primary IS NOT NULL
GROUP BY categories.primary
ORDER BY count DESC
LIMIT 100
"""

tn_categories = con.execute(explore_categories).fetchdf()
print("Top categories in Tennessee:")
print(tn_categories.to_string())

# Search for categories containing our keywords
print("\n\nSearching for related categories:")
keywords = ['mobile', 'home', 'shelter', 'social', 'nursing', 'assisted', 'living']
for keyword in keywords:
    matches = tn_categories[tn_categories['category'].str.contains(keyword, case=False, na=False)]
    if len(matches) > 0:
        print(f"\n'{keyword}' matches:")
        print(matches.to_string())

In [7]:
# Count by category
category_counts = df['category'].value_counts()
print("\nCount by Category:")
print("-" * 40)
print(category_counts)


Count by Category:
----------------------------------------
category
mobile_home_park    123
homeless_shelter     52
Name: count, dtype: int64


In [8]:
# Display full results
df

,id,name,category,state,city,geometry
0,e05479f8-c1ab-4d58-9081-b571154f279d,Candlelight,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,75d242ff-9944-494a-b1c4-3d9407c47972,Wheel Estates,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,98c085eb-0449-462e-ada1-8d9af1851092,Candlewood,mobile_home_park,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,27471146-80fa-405a-a7d7-6ed90d8548f8,Housing The Homeless - Helps,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,70fc56d2-7c5e-4759-8240-1fd9bfc663ed,Calvary Rescue Mission,homeless_shelter,TN,Memphis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...
170,f91a2adc-afc0-423b-9da0-1688250e4f6f,River's Edge,mobile_home_park,TN,Hampton,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
171,790e1aaa-6deb-4af1-b44c-deda06864fc0,M H Canter Mobile Home Park,mobile_home_park,TN,Elizabethton,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
172,02ee8032-b221-4447-8cf6-c0f398ea69f5,Dogwood Acres,mobile_home_park,TN,Bristol,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
173,125bd40f-14e1-405b-b72f-1094c4129dc7,Bristol Heights Mobile Home Estates,mobile_home_park,TN,Bristol,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [9]:
# Optional: Export to CSV
# df.to_csv('tennessee_filtered_places.csv', index=False)
# print("Data exported to tennessee_filtered_places.csv")

In [10]:
# Optional: Convert geometry to lat/lon for easier visualization (Tennessee only)
geo_query = """
SELECT 
    id, 
    names.primary as name, 
    categories.primary as category,
    addresses[1].locality as city,
    ST_X(geometry) as longitude,
    ST_Y(geometry) as latitude
FROM places
WHERE categories.primary IN (
    'mobile_home_park',
    'homeless_shelter', 
    'social_facility', 
    'nursing_home', 
    'assisted_living'
)
AND addresses[1].region = 'TN'
"""

df_geo = con.execute(geo_query).fetchdf()
print(f"Found {len(df_geo):,} places in Tennessee with coordinates")
df_geo.head(10)

Found 175 places in Tennessee with coordinates


,id,name,category,city,longitude,latitude
0,e05479f8-c1ab-4d58-9081-b571154f279d,Candlelight,mobile_home_park,Memphis,-90.073826,35.032050
1,75d242ff-9944-494a-b1c4-3d9407c47972,Wheel Estates,mobile_home_park,Memphis,-90.013582,35.065114
2,98c085eb-0449-462e-ada1-8d9af1851092,Candlewood,mobile_home_park,Memphis,-90.021032,35.073195
3,27471146-80fa-405a-a7d7-6ed90d8548f8,Housing The Homeless - Helps,homeless_shelter,Memphis,-90.031354,35.110407
4,70fc56d2-7c5e-4759-8240-1fd9bfc663ed,Calvary Rescue Mission,homeless_shelter,Memphis,-90.054363,35.121126
5,31af2467-638b-4c12-a809-3efa567a9079,Barron Heights Transitional Center,homeless_shelter,Memphis,-90.015738,35.125443
6,b9c946cf-6eb3-475a-9936-33c666d166cd,Alpha Omega Veterans Services - Vinton,homeless_shelter,Memphis,-90.019516,35.131016
7,a33313a3-f6a3-43e2-9550-c5c7b56cfe10,Dorothy Day House Of Hospitality,homeless_shelter,Memphis,-90.022231,35.133037
8,25898f67-868f-4417-9420-f4811013ac1f,Dorothy Day House,homeless_shelter,Memphis,-90.012910,35.143462
9,ff70aa3c-6dac-4dfb-bafa-f0bec89a0ed9,Room In The Inn,homeless_shelter,Memphis,-90.029432,35.151479


In [11]:
# Export data to CSV
output_file = 'tennessee_places.csv'

# Export the geo dataframe (with lat/lon coordinates)
df_geo.to_csv(output_file, index=False)

print(f"✓ Exported {len(df_geo):,} places to {output_file}")
print(f"\nColumns included: {', '.join(df_geo.columns.tolist())}")

✓ Exported 175 places to tennessee_places.csv

Columns included: id, name, category, city, longitude, latitude


In [12]:
# Close connection when done
con.close()
print("Connection closed")

Connection closed
